In [1]:
import sqlite3
import pandas as pd

df = pd.read_csv("../data/cleaned/engineered_telco_customer_churn.csv")
conn = sqlite3.connect("../data/cleaned/customer_churn.db")
df.to_sql("customer_churn", conn, if_exists="replace", index=False)
conn.close()
print("DataFrame saved to SQLite database successfully.") 

DataFrame saved to SQLite database successfully.


## 2. Preview Data in SQL

In [3]:
conn = sqlite3.connect("../data/cleaned/customer_churn.db")
pd.read_sql("SELECT * FROM customer_churn LIMIT 5", conn)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TenureBucket_Mid-Term,TenureBucket_Long-Term,NumServices,MonthlyCharges_Tenure,MonthlyCharges_per_Service
0,0,0,1,0,1,0,1,29.85,29.85,0,...,0,0,0,1,0,0,0,1,29.85,29.849702
1,1,0,0,0,34,1,0,56.95,1889.50,0,...,1,0,0,0,1,0,1,3,1936.30,18.983270
2,1,0,0,0,2,1,1,53.85,108.15,1,...,0,0,0,0,1,0,0,3,107.70,17.949940
3,1,0,0,0,45,0,0,42.30,1840.75,0,...,1,0,0,0,0,0,1,3,1903.50,14.099953
4,0,0,0,0,2,1,1,70.70,151.65,1,...,0,0,0,1,0,0,0,1,141.40,70.699293


## 4. Evaluating overall Churn rate

In [6]:
pd.read_sql_query("""
SELECT
                  COUNT(*) AS total_customers,
                  SUM(CASE WHEN churn = 1 THEN 1 ELSE 0 END) AS churned_customers,
                  ROUND(SUM(CASE WHEN churn = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS churn_rate_percentage
                  FROM customer_churn
                  """, conn

)

,total_customers,churned_customers,churn_rate_percentage
0,7043,1869,26.54


From this we can see that roughly 26.54 of Telco's clients have churned. We will dive deeper into this group by evaluating various characteristics, namely: Contract Type, Tenure, and Internet Service Subscription

### Distribution of Contract types

In [13]:
df_contract = pd.read_sql_query("""
SELECT 
    CASE
        WHEN "Contract_One year" = 1 THEN 'One year'
        WHEN "Contract_Two year" = 1 THEN 'Two year' 
        ELSE 'Month-to-month'
    END AS contract_type,
    COUNT(*) AS total_customers,
    ROUND(COUNT(*)/7043.0 * 100, 2) AS percentage_of_customers
FROM customer_churn
GROUP BY contract_type
ORDER BY total_customers DESC
""", conn)

df_contract.to_csv("../data/SQL_queries/contract_summary.csv", index=False)


In [14]:
df_contract

,contract_type,total_customers,percentage_of_customers
0,Month-to-month,3875,55.02
1,Two year,1695,24.07
2,One year,1473,20.91


In this table we can see that more than double of Telco's client base is with the company on a month-to-month basis. We will compare the amount of clients that end up leaving by contract type.

### Churn By Contract type

In [ ]:
df_contract_churn = pd.read_sql_query(
    """
    SELECT
    SUM(CASE WHEN "Contract_One year" = 1 THEN 1 ELSE 0 END) AS one_year_contracts,
    SUM(CASE WHEN "Contract_Two year" = 1 THEN 1 ELSE 0 END) AS two_year_contracts,
    SUM(CASE WHEN ("Contract_One year" = 0 AND "Contract_Two year" = 0) = 1 THEN 1 ELSE 0 END) AS month_to_month_contracts
    FROM customer_churn
    WHERE churn = 1
    """, conn
)
df_contract_churn.to_csv("../data/SQL_queries/churn_contract_summary.csv", index=False)


In [11]:
df_contract_churn

,one_year_contracts,two_year_contracts,month_to_month_contracts
0,166,48,1655


From this query we that an overwhelming majority Telco's Churners (88.5%) arise from clients on month-to-month type contracts.

### Average Tenure of churners

In [ ]:
df_tenure = pd.read_sql_query(
    """
    SELECT
    CASE
        WHEN tenure BETWEEN 0 AND 6 THEN '0-6 months'
        WHEN tenure BETWEEN 7 AND 12 THEN '7-12 months'
        WHEN tenure BETWEEN 13 AND 24 THEN '13-24 months'
        WHEN tenure BETWEEN 25 AND 36 THEN '25-36 months'
        WHEN tenure BETWEEN 37 AND 48 THEN '37-48 months'
        WHEN tenure BETWEEN 49 AND 60 THEN '49-60 months'
        ELSE 'Over 60 months'
    END AS tenure_group,
    COUNT(*) AS customer_count,
    SUM(CASE WHEN churn = 1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN churn = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS churn_rate
    FROM customer_churn
    GROUP BY tenure_group
    ORDER BY churn_rate DESC
    """, conn
)

df_tenure.to_csv("../data/SQL_queries/churn_tenure_summary.csv", index=False)

In [6]:
df_tenure

,tenure_group,customer_count,churned_customers,churn_rate
0,0-6 months,1481,784,52.94
1,7-12 months,705,253,35.89
2,13-24 months,1024,294,28.71
3,25-36 months,832,180,21.63
4,37-48 months,762,145,19.03
5,49-60 months,832,120,14.42
6,Over 60 months,1407,93,6.61


From this table we can that we haven't had issue acquiring new clients however are clearly failing to retain them with 53% of new clients leaving after just 6 months with Telco. 

### Distribution of clients by whether they have one service or not:

In [25]:
df_service = pd.read_sql_query(
    """
    SELECT
    CASE 
    WHEN NumServices = 1 THEN 'One Service'
    ELSE  'Multiple Services'
    END AS service_group,
    Churn,
    COUNT(*) AS customer_count
    FROM customer_churn
    GROUP BY service_group, Churn
    ORDER BY service_group DESC
    """, conn)


In [26]:
df_service

,service_group,Churn,customer_count
0,One Service,0,1342
1,One Service,1,359
2,Multiple Services,0,3832
3,Multiple Services,1,1510


### Average Monthly Charges by Tenure


In [30]:
df_avg_charges = pd.read_sql_query("""SELECT
    tenure,
    CASE
        WHEN NumServices = 1 THEN 'One Service'
        ELSE 'Multiple Services'
    END AS service_group,
    Churn,
    AVG(MonthlyCharges) AS avg_monthly_charges,
    COUNT(*) AS customer_count
FROM customer_churn
GROUP BY
    tenure,
    service_group,
    Churn
ORDER BY tenure ASC, service_group DESC, Churn ASC
""", conn)

df_avg_charges.to_csv("../data/SQL_queries/avg_charges_by_tenure.csv", index=False)

In [29]:
df_avg_charges

,tenure,service_group,Churn,avg_monthly_charges,customer_count
0,0,One Service,0,19.950000,4
1,0,Multiple Services,0,53.685714,7
2,1,One Service,0,29.398344,151
3,1,One Service,1,46.701099,182
4,1,Multiple Services,0,53.581098,82
...,...,...,...,...,...
255,71,Multiple Services,0,79.086735,147
256,71,Multiple Services,1,95.133333,6
257,72,One Service,0,20.075000,26
258,72,Multiple Services,0,85.177879,330


### Are above-average spenders more likely to churn?

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("../data/cleaned/customer_churn.db")

df_summary = pd.read_sql_query("""WITH charges AS (
    SELECT 
        CASE
            WHEN "Contract_One year" = 1 THEN 'One year'
            WHEN "Contract_Two year" = 1 THEN 'Two year'
            ELSE 'Month-to-month'
        END AS contract_type,
        Churn,
        MonthlyCharges,
        CASE 
            WHEN MonthlyCharges > AVG(MonthlyCharges) OVER () THEN 'Above Average'
            ELSE 'Below Average'
        END AS charge_flag
    FROM customer_churn
)
SELECT 
    contract_type AS Contract,
    charge_flag,
    COUNT(*) AS num_customers,
    SUM(CASE WHEN Churn = 1 THEN 1 ELSE 0 END) AS churned,
    ROUND(
        100.0 * SUM(CASE WHEN Churn = 1 THEN 1 ELSE 0 END) 
        / COUNT(*),
        2
    ) AS churn_rate_pct
FROM charges
GROUP BY contract_type, charge_flag
ORDER BY contract_type, charge_flag;

""",conn
)
df_summary.to_csv("../data/SQL_queries/churn_pricing_summary.csv", index=False)

In [4]:
df_summary

,Contract,charge_flag,num_customers,churned,churn_rate_pct
0,Month-to-month,Above Average,2300,1187,51.61
1,Month-to-month,Below Average,1575,468,29.71
2,One year,Above Average,785,130,16.56
3,One year,Below Average,688,36,5.23
4,Two year,Above Average,838,38,4.53
5,Two year,Below Average,857,10,1.17


Among month-to-month customers — the group most vulnerable to churn — those paying above-average monthly charges churn at a rate of 51.6%, compared to 29.7% for those paying below average.

This strongly suggests that pricing pressure is a primary churn driver in the short-term contract segment.